In [19]:
import os
import re
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [21]:
def create_dataframe(data, columns):
    """
    Erstellt ein pandas DataFrame aus einer Liste von Listen und einer Liste von Spaltennamen.

    :param data: List[List], die Datenreihen
    :param columns: List[str], die Spaltennamen
    :return: pandas.DataFrame
    """
    return pd.DataFrame(data, columns=columns)

data = [
    ['degewo AG', 'https://www.degewo.de/presse', 'Konzernlagebericht', 'text'],
    ['GESOBAU AG', 'https://www.gesobau.de/ueber-uns/gesobau-in-zahlen/geschaeftsbericht/', r'Gesch.{1,2}ftsbericht', 'url']    
]

columns = ["Unternehmen", "URL", "Stichwort", 'Suchbereich']

df = create_dataframe(data, columns)
print(df)

  Unternehmen                                                URL  \
0   degewo AG                       https://www.degewo.de/presse   
1  GESOBAU AG  https://www.gesobau.de/ueber-uns/gesobau-in-za...   

               Stichwort Suchbereich  
0     Konzernlagebericht        text  
1  Gesch.{1,2}ftsbericht         url  


In [25]:
def download_pdfs(url, DOWNLOAD_FOLDER, pattern, link_part = 'text', verbose = False):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all("a", href=True)

    count_downloaded = 0
    count_skipped = 0
    count_error = 0
    
    for link in links:
        if link_part == 'text':
            search_string = link.text
        elif link_part == 'url':
            search_string = link['href']
        if re.search(pattern, search_string) and link["href"].endswith(".pdf"):
            pdf_url = link["href"]
            if not pdf_url.startswith("http"):
                pdf_url = requests.compat.urljoin(url, pdf_url)
            
            pdf_name = os.path.join(DOWNLOAD_FOLDER, pdf_url.split("/")[-1])

            if os.path.exists(pdf_name):
                if verbose:
                    print(f"Bereits vorhanden: {pdf_name}")
                
                count_skipped += 1
                continue
            
            if verbose:
                print(f"Lade herunter: {pdf_url}")

            pdf_response = requests.get(pdf_url)
            with open(pdf_name, "wb") as file:
                file.write(pdf_response.content)
                count_downloaded += 1
                
                if verbose:
                    print(f"Gespeichert als: {pdf_name}")

    if not verbose:
        print(f'Es wurden {count_downloaded} Berichte heruntergeladen.\nEs wurden {count_skipped} schon vorhandene Berichte übersprungen.\nEs gab {count_error} Fehler.')

In [31]:
def pdf_downloader_loop(df, download_parent_folder = 'Geschaeftsberichte/'):
    for index, row in df.iterrows():
        unternehmen = row['Unternehmen']
        URL = row['URL']
        stichwort = row['Stichwort']
        suchbereich = row['Suchbereich']

        # Verzeichnis für die heruntergeladenen PDFs
        DOWNLOAD_FOLDER = download_parent_folder+unternehmen
        verbose = False if os.path.exists(DOWNLOAD_FOLDER) else True
        os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

        print(f'\n####### {unternehmen} #######')
        download_pdfs(URL, DOWNLOAD_FOLDER, stichwort, suchbereich, verbose)
        file_count = len([file for file in os.listdir(DOWNLOAD_FOLDER) if file.endswith('.pdf')])
        print(f'Es befinden sich {file_count} Berichte im Verzeichnis.')

pdf_downloader_loop(df)


####### degewo AG #######
Es wurden 0 Berichte heruntergeladen.
Es wurden 7 schon vorhandene Berichte übersprungen.
Es gab 0 Fehler.
Es befinden sich 7 Berichte im Verzeichnis.

####### GESOBAU AG #######
Es wurden 0 Berichte heruntergeladen.
Es wurden 24 schon vorhandene Berichte übersprungen.
Es gab 0 Fehler.
Es befinden sich 12 Berichte im Verzeichnis.
